In [ ]:
import os
import pymysql
from typing import List

from alibabacloud_sls20201230.client import Client as Sls20201230Client
from alibabacloud_tea_openapi import models as open_api_models
from alibabacloud_sls20201230 import models as sls_20201230_models
from alibabacloud_tea_util import models as util_models
from alibabacloud_tea_util.client import Client as UtilClient
import os
import pandas as pd
from datetime import datetime, timedelta

import logging

# Configure the logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

ALIBABA_CLOUD_ACCESS_KEY_ID = os.environ["ALIBABA_CLOUD_ACCESS_KEY_ID"]
ALIBABA_CLOUD_ACCESS_KEY_SECRET = os.environ["ALIBABA_CLOUD_ACCESS_KEY_SECRET"]


MYSQL_HOST = "mysql-xm.summerfarm.net"
MYSQL_PORT = 3308
MYSQL_USER_NAME = "qa"
MYSQL_PASSWORD = "xianmu619"

config = open_api_models.Config(
    access_key_id=ALIBABA_CLOUD_ACCESS_KEY_ID,
    access_key_secret=ALIBABA_CLOUD_ACCESS_KEY_SECRET,
    read_timeout=120 * 1000,
    connect_timeout=10 * 1000,
)
# Endpoint 请参考 https://api.aliyun.com/product/Sls
config.endpoint = f"cn-hangzhou.log.aliyuncs.com"
sls_client = Sls20201230Client(config)

# Create a connection to the MySQL server
conn = pymysql.connect(
    host=MYSQL_HOST,
    port=MYSQL_PORT,
    user=MYSQL_USER_NAME,
    password=MYSQL_PASSWORD,
    database="xianmudb",
)

import random

def generate_random_numerical_string(length=12):
    """Generate a random numerical string of a specified length."""
    if length <= 0:
        raise ValueError("Length must be a positive integer")
    str_generated= ''.join(random.choices('0123456789', k=length))
    return f'01000{str_generated}'

print(generate_random_numerical_string())


In [ ]:
print("""'null'""".replace("\'null\'", "null"))

In [ ]:
def write_data_with_sql(sql):
    global conn
    # Create a cursor object
    cursor = conn.cursor()
    # Execute the SQL query for each row in the DataFrame
    try:
        sql = sql.replace("\'null\'", "null")
        sql = sql.replace("None", "null")
        sql = sql.replace(",nan,", ",null,")
        cursor.execute(sql)
        conn.commit()
    except Exception as e:
        logging.info(f"sql:{sql}\nError inserting data: {e}")
        conn.rollback()
        raise e
    else:
        cursor.close()


def write_df_to_mysql(
    df: pd.DataFrame,
    table_name="product_views",
    columns=[
        "minute_of_day",
        "day",
        "time_of_day",
        "pdid",
        "sku",
        "name",
        "views",
        "avg_sale_price",
        "max_sale_price",
        "min_sale_price",
        "viewed_users",
    ],
    colomns_to_update=[
        "views",
        "avg_sale_price",
        "max_sale_price",
        "min_sale_price",
        "viewed_users",
    ],
) -> any:
    df = df[columns]
    update_colomns = [f"{col}=values({col})" for col in colomns_to_update]
    update_colomns = ",\n".join(update_colomns)
    sql_template = f"""INSERT INTO {table_name}({','.join(columns)}) \nvalues __VALUES__ ON DUPLICATE KEY UPDATE 
    {update_colomns};"""
    values_list = []
    for index, row in df.iterrows():
        value_of_row = []
        for col in columns:
            col_value_without_quote = f"{row[col]}".replace("'", "\\'")
            value_of_row.append(f"'{col_value_without_quote}'")
        values_list.append(f"""({','.join(value_of_row)})""")
        if index % 200 == 0:
            sql = sql_template.replace("__VALUES__", ",\n".join(values_list))
            # logging.info(sql)
            write_data_with_sql(sql=sql)
            values_list = []
            # return "TESTED"
    if len(values_list) > 0:
        sql = sql_template.replace("__VALUES__", ",\n".join(values_list))
        # logging.info(sql)
        write_data_with_sql(sql=sql)

## 获取订单数据

In [ ]:
import pymysql


# Function to establish a database connection
def connect_to_db():
    return pymysql.connect(
        host=MYSQL_HOST,
        user=MYSQL_USER_NAME,
        password=MYSQL_PASSWORD,
        port=3308,
        db="xianmudb",
        charset="utf8mb4",
        cursorclass=pymysql.cursors.DictCursor,
    )

In [ ]:
order_item_col = [
    "pd_name",
    "sku",
    "weight",
    "maturity",
    "order_no",
    "category_id",
    "amount",
    "price",
    "original_price",
    "picture_path",
    "add_time",
    "storage_location",
    "suit_id",
    "suit_name",
    "status",
    "m_price",
    "volume",
    "weight_num",
    "total_price",
    "product_type",
    "create_time",
    "actual_total_price",
    "update_time",
    "sku_name",
    "info",
]

order_col = [
    "order_no",
    "m_id",
    "order_time",
    "type",
    "status",
    "delivery_fee",
    "total_price",
    "remark",
    "out_times",
    "discount_type",
    "out_times_fee",
    "area_no",
    "m_size",
    "account_id",
    "origin_price",
    "out_stock",
    "order_sale_type",
    "receivable_status",
    "invoice_status",
    "order_pay_type",
]

# Establish the connection
for index in range(1, 20):
    connection = connect_to_db()
    order_df = None
    order_item_df = None
    try:
        with connection.cursor() as cursor:
            query = "SELECT * FROM orders where status in (2,3,6) and order_no not like '01000%' order by order_id desc limit 1000"
            cursor.execute(query)
            rows = cursor.fetchall()
            order_df = pd.DataFrame(rows)
            order_no_list = order_df["order_no"].unique().tolist()
            order_no_list_str = "','".join(order_no_list)
            sql = f"select * from order_item where order_no in ('{order_no_list_str}')"

        with connection.cursor() as cursor:
            cursor.execute(sql)
            rows = cursor.fetchall()
            order_item_df = pd.DataFrame(rows)
    finally:
        connection.close()

    order_no_map = {}
    for order_no in order_no_list:
        order_no_map[order_no] = generate_random_numerical_string()

    def replace_order_no_with_random(order_no):
        return order_no_map.get(order_no)

    order_df["order_no"] = order_df["order_no"].apply(replace_order_no_with_random)
    order_df["m_id"] = 349548
    order_df["area_no"] = 29380
    order_df["account_id"] = 8881
    order_df["order_pay_type"] = 2
    order_item_df["order_no"] = order_df["order_no"].apply(replace_order_no_with_random)
    order_item_df["info"] = "唐鹏随机生成的订单"
    order_item_df["m_price"] = 0.5
    order_item_df["update_time"] = datetime.now()
    order_item_df["product_type"] = 0
    order_item_df["actual_total_price"] = 100.00

    write_df_to_mysql(
        order_df, table_name="orders", columns=order_col, colomns_to_update=["area_no"]
    )
    write_df_to_mysql(
        order_item_df,
        table_name="order_item",
        columns=order_item_col,
        colomns_to_update=["info"],
    )

In [2]:
import logging

# Configure the logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

logging.info("萨大王的")

2024-06-21 19:15:52 - INFO - 萨大王的
